In [30]:
################################################################
## This notebook is reproducing results from https://github.com/kanekomasahiro/gp_debias
## You should upload debiasing.ipynb into https://colab.research.google.com/ to run it
################################################################
!git clone https://github.com/pavelbystrov1/debiasing_evaluation
!git clone https://github.com/kanekomasahiro/gp_debias

fatal: destination path 'debiasing_evaluation' already exists and is not an empty directory.
fatal: destination path 'gp_debias' already exists and is not an empty directory.


In [3]:
!cd ./gp_debias; mkdir d; cd d; git clone https://github.com/uclanlp/gn_glove.git
!paste ./gp_debias/d/gn_glove/wordlist/female_word_file.txt ./gp_debias/d/gn_glove/wordlist/male_word_file.txt > ./gp_debias/wordlist/gender_pair.tsv 
!cp -r ./gp_debias/d/gn_glove/SemBias ./gp_debias/
!ls ./gp_debias

mkdir: cannot create directory ‘d’: File exists
fatal: destination path 'gn_glove' already exists and is not an empty directory.
d	     eval_word_embeddings.py  LICENSE	 run.sh   src
download.sh  hyperparams	      README.md  SemBias  wordlist


In [4]:
##############################################################################
# Glove embedding trained on wikidump dataset can be downloaded using code below or manually - https://drive.google.com/open?id=1jrbQmpB5ZNH4w54yujeAvNFAfVEG0SuE
# GN-Glove embedding trained on wikidump dataset can be downloaded using code below or manually - https://drive.google.com/file/d/1v82WF43w-lE-vpZd0JC1K8WYZQkTy_ii/
##############################################################################
!gdown --id '1jrbQmpB5ZNH4w54yujeAvNFAfVEG0SuE'
!gdown --id '1v82WF43w-lE-vpZd0JC1K8WYZQkTy_ii'


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1jrbQmpB5ZNH4w54yujeAvNFAfVEG0SuE
To: /content/GloVe.zip
100% 368M/368M [00:04<00:00, 83.7MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1v82WF43w-lE-vpZd0JC1K8WYZQkTy_ii
To: /content/GN-GloVe-L1-0.8-0.8.txt.zip
100% 369M/369M [00:05<00:00, 73.4MB/s]


In [5]:
##########################################
# unzip GloVe embedding
##########################################
!cd gp_debias; mkdir -p src/glove_model
!cp GloVe.zip ./gp_debias/src/glove_model/
!cd gp_debias/src/glove_model; unzip GloVe.zip
!cd gp_debias/src/glove_model; sed -i "1s/^/322636 300\n/" vectors.txt
!cd gp_debias/; mkdir embeddings
!mv gp_debias/src/glove_model/vectors.txt gp_debias/embeddings/glove.txt
!cd gp_debias; mkdir -p src/debiased_glove
!ls gp_debias/embeddings/


Archive:  GloVe.zip
  inflating: vectors.txt             
glove.txt


In [6]:
###########################################
# create classifiers for glove
###########################################
!cd gp_debias; python src/train.py glove

Loading word embedding
Loading data
Pre-training autoencoder
/content/gp_debias/src/pre_train_autoencoder.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  eval_inputs = torch.split(torch.FloatTensor(emb_list[:hp.pta_dev_num]), hp.pta_batch_size)
src/train.py:426: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  train_female_embs = [encoder(torch.FloatTensor(emb[word[0]]).cuda()).data if hp.gpu >= 0 else encod

In [8]:
###########################################
# save gp_glove embedding
###########################################
!cd gp_debias; python src/eval.py glove
!ls gp_debias/src/debiased_glove

Generating emb...
Start generating
src/eval.py:95: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  inputs = torch.split(torch.stack([torch.FloatTensor(w2v[word]) for word in w2v.vocab.keys()]), 1024)
SemBias
definition: 0.8386363636363636
stereotype: 0.08181818181818182
none: 0.07954545454545454
sub definition: 0.625
sub stereotype: 0.15
sub none: 0.225
Saving emb...
gender_debiased.bin  gender_debiased.txt


In [9]:
#############################################################
# creating folder for evaluation of generated word embeddings
# and copy glove.txt and gp_glove.txt there
#############################################################
!cd ./debiasing_evaluation; mkdir glove
!mv gp_debias/embeddings/glove.txt debiasing_evaluation/glove/glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.txt debiasing_evaluation/glove/gp_glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.bin debiasing_evaluation/glove/gp_glove.bin


In [20]:
#######################################
#      prepare data for GN-Glove
#######################################
!cd gp_debias; 
!cp /content/GN-GloVe-L1-0.8-0.8.txt.zip /content/gp_debias/src/glove_model/
!cd gp_debias/src/glove_model; unzip "GN-GloVe-L1-0.8-0.8.txt.zip"
!cd gp_debias/src/glove_model; sed -i "1s/^/322636 300\n/" vectors300.txt
!cp gp_debias/src/glove_model/vectors300.txt gp_debias/src/glove_model/glove.txt
!mv gp_debias/src/glove_model/glove.txt gp_debias/embeddings/glove.txt
!ls gp_debias/src/glove_model
!cd gp_debias/src/debiased_glove; rm gender_debiased.txt gender_debiased.bin

best_model.pt  GloVe.zip  GN-GloVe-L1-0.8-0.8.txt.zip  __MACOSX  vectors300.txt
rm: cannot remove 'gender_debiased.txt': No such file or directory
rm: cannot remove 'gender_debiased.bin': No such file or directory


In [21]:
###########################################
# create classifiers for GN-glove and save gp_gn_glove embedding
###########################################
!cd gp_debias; python src/train.py glove
!cd gp_debias; python src/eval.py glove
!ls gp_debias/src/debiased_glove

Loading word embedding
Loading data
Pre-training autoencoder
/content/gp_debias/src/pre_train_autoencoder.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  eval_inputs = torch.split(torch.FloatTensor(emb_list[:hp.pta_dev_num]), hp.pta_batch_size)
src/train.py:426: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  train_female_embs = [encoder(torch.FloatTensor(emb[word[0]]).cuda()).data if hp.gpu >= 0 else encod

In [23]:
#############################################################
# copy GN-glove and gp_gn_glove there
#############################################################
!mv gp_debias/embeddings/glove.txt debiasing_evaluation/glove/gn_glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.txt debiasing_evaluation/glove/gp_gn_glove.txt
!mv gp_debias/src/debiased_glove/gender_debiased.bin debiasing_evaluation/glove/gp_gn_glove.bin
!ls debiasing_evaluation/glove/

glove.txt     gp_glove.bin  gp_gn_glove.bin
gn_glove.txt  gp_glove.txt  gp_gn_glove.txt


In [32]:
###############################################################
# load SimLex, datasets from Semantic-measure-assessment study and Google anthologies
# and copy test dataset for evaluation
###############################################################
!wget https://fh295.github.io/SimLex-999.zip
!git clone https://github.com/MohamedAliHadjTaieb/Semantic-measure-assessment-review-study
!wget http://download.tensorflow.org/data/questions-words.txt
!unzip SimLex-999.zip
!mv questions-words.txt debiasing_evaluation/glove/google_anthology_ds.txt
!mv SimLex-999/SimLex-999.txt debiasing_evaluation/glove/SimLex-999.txt
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Relatedness/MT771.csv" debiasing_evaluation/glove/MTurk.csv
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Relatedness/RW2034.csv" debiasing_evaluation/glove/
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Relatedness/WordSim353.csv" debiasing_evaluation/glove/
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Similarity/RG65.csv" debiasing_evaluation/glove/
!cp "Semantic-measure-assessment-review-study/DataSets/English/Semantic Similarity/MEN3000.csv" debiasing_evaluation/glove/

In [34]:
############################################
#            Preparation DONE
############################################
!ls debiasing_evaluation/glove/

glove.txt		 gp_glove.txt	  MTurk.csv	  WordSim353.csv
gn_glove.txt		 gp_gn_glove.bin  RG65.csv
google_anthology_ds.txt  gp_gn_glove.txt  RW2034.csv
gp_glove.bin		 MEN3000.csv	  SimLex-999.txt
